<a href="https://colab.research.google.com/github/Jeremy-su1/ai-algorithm/blob/main/Singlelable_test_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torch
!pip install transformers
!pip install peft
!pip install huggingface_hub
!pip install evaluate

In [2]:
from google.colab import userdata
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType  # PEFT 라이브러리 필요
import pandas as pd
from google.colab import drive
from sklearn.metrics import f1_score

In [4]:
import pandas as pd

#drive.mount('/content/drive', force_remount=False)
test_path = ('/content/drive/MyDrive/DataSet_new/stackexchange_test_dataset_1000_title.csv')
df_test = pd.read_csv(test_path)
num_labels= len(df_test['label'].unique())

#df_test
text_test= list(df_test['title_content'])
label_test = list(df_test['label'])
#test_dataset = generate_dataset(text_test, label_test)

In [15]:
from peft import PeftModel, LoraConfig, get_peft_model
drive.mount('/content/drive', force_remount=False)

token =userdata.get('Hugging_Face_llama3.2')
model_path = 'meta-llama/Llama-3.2-1B'

num_labels = num_labels
base_model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=num_labels, use_auth_token=token)


tokenizer = AutoTokenizer.from_pretrained(model_path, use_auth_token=token)
tokenizer.pad_token = tokenizer.eos_token
base_model.config.pad_token_id = tokenizer.pad_token_id

#lora_path = "/content/drive/MyDrive/single_label/checkpoint_6250"  # Adjust this path as needed

#model = PeftModel.from_pretrained(base_model, lora_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [16]:
import torch

# ... (your existing code) ...

# 모델을 평가 모드로 설정
model = base_model

model.eval()
model.to('cuda')  # 모델을 명시적으로 GPU로 이동
#base_model.eval()
#base_model.to('cuda')  # 모델을 명시적으로 GPU로 이동


test_encodings = tokenizer(text_test, truncation=True, padding=True, max_length=100, return_tensors="pt")

# 예측 수행
predictions = []
batch_size = 16  # GPU 메모리에 따라 조정
num_batches = len(test_encodings['input_ids']) // batch_size + (len(test_encodings['input_ids']) % batch_size != 0)

with torch.no_grad():
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(test_encodings['input_ids']))

        # 각 배치를 GPU로 이동
        batch_encodings = {k: v[start_idx:end_idx].to('cuda') for k, v in test_encodings.items()}

        # 모델 출력 및 예측 수행
        outputs = model(**batch_encodings)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)  # 소프트맥스 함수로 확률 변환
        preds = torch.argmax(probs, dim=1)  # 가장 높은 확률의 클래스 선택
        predictions.extend(preds.cpu().numpy())  # 결과를 CPU로 이동하여 numpy 배열에 추가

In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import evaluate

accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics_test(predictions, labels):

    metrics = {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"],
        "macro_precision": precision_metric.compute(predictions=predictions, references=labels, average="macro")["precision"],
        "macro_recall": recall_metric.compute(predictions=predictions, references=labels, average="macro")["recall"],
        "macro_f1": f1_metric.compute(predictions=predictions, references=labels, average="macro")["f1"],
        "micro_precision": precision_metric.compute(predictions=predictions, references=labels, average="micro")["precision"],
        "micro_recall": recall_metric.compute(predictions=predictions, references=labels, average="micro")["recall"],
        "micro_f1": f1_metric.compute(predictions=predictions, references=labels, average="micro")["f1"],
        "weighted_precision": precision_metric.compute(predictions=predictions, references=labels, average="weighted")["precision"],
        "weighted_recall": recall_metric.compute(predictions=predictions, references=labels, average="weighted")["recall"],
        "weighted_f1": f1_metric.compute(predictions=predictions, references=labels, average="weighted")["f1"],
    }
    return metrics

In [18]:
#test_result =compute_metrics_test(predictions, label_test)
test_result_2 =compute_metrics_test(predictions, label_test)

In [19]:
##Baseline result
print(test_result_2)

{'accuracy': 0.17, 'macro_precision': 0.13426821732430025, 'macro_recall': 0.17, 'macro_f1': 0.1431252763684701, 'micro_precision': 0.17, 'micro_recall': 0.17, 'micro_f1': 0.17, 'weighted_precision': 0.13426821732430022, 'weighted_recall': 0.17, 'weighted_f1': 0.1431252763684701}


### Trained result

In [11]:
print(test_result)

{'accuracy': 0.978, 'macro_precision': 0.9782915145403924, 'macro_recall': 0.978, 'macro_f1': 0.9779777245401144, 'micro_precision': 0.978, 'micro_recall': 0.978, 'micro_f1': 0.978, 'weighted_precision': 0.9782915145403925, 'weighted_recall': 0.978, 'weighted_f1': 0.9779777245401144}
